<a href="https://colab.research.google.com/github/sr606/Machine_Learning_CaseStudies/blob/main/OLA_Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings("ignore")

In [4]:
ola = pd.read_csv("/content/drive/MyDrive/Files/ola_driver_scaler.csv")

In [5]:
ola.head()

,Unnamed: 0,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,0,01/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,1,02/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,2,03/01/19,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,3,11/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
4,4,12/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1


#**Exploratory Data Analysis**

In [6]:
ola.shape

(19104, 14)

In [7]:
print("Rows in the ola dataset:", ola.shape[0])
print("Columns in the ola dataset:", ola.shape[1])

Rows in the ola dataset: 19104
Columns in the ola dataset: 14


In [8]:
ola.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            19104 non-null  int64  
 1   MMM-YY                19104 non-null  object 
 2   Driver_ID             19104 non-null  int64  
 3   Age                   19043 non-null  float64
 4   Gender                19052 non-null  float64
 5   City                  19104 non-null  object 
 6   Education_Level       19104 non-null  int64  
 7   Income                19104 non-null  int64  
 8   Dateofjoining         19104 non-null  object 
 9   LastWorkingDate       1616 non-null   object 
 10  Joining Designation   19104 non-null  int64  
 11  Grade                 19104 non-null  int64  
 12  Total Business Value  19104 non-null  int64  
 13  Quarterly Rating      19104 non-null  int64  
dtypes: float64(2), int64(8), object(4)
memory usage: 2.0+ MB


**Column Profiling:**



1.   MMM - YY: Reporting Date (Monthly) (date-time)
2.   Age: Age of the employee(numerical)
3. Gender: Gender of the employee(categorical)
4. City: City Code of the employee(categorical)
5. Education_Level: Education level - 0 for 10+ , 1 for 12+, 2 for graduate (categorical)

6. Income : Monthly average Income of the employee (numerical)

7.  Date Of Joining : Joining date for the employee (date-time)

8. LastWorkingDate : Last date of working for the employee - Target Feature (date-time, but will be converted to categorical)

9. Joining Designation : Designation of the employee at the time of joining (categorical, ordinal)

10.  Grade : Grade of the employee at the time of reporting (categorical, ordinal)

11. Total Business Value : The total business value acquired by the employee in a month (negative business indicates cancellation/refund or car EMI adjustments) (numerical)

12.  Quarterly Rating : Quarterly rating of the employee: 1,2,3,4,5 (categorical, ordinal - higher is better)



In [9]:
ola.describe()

,Unnamed: 0,Driver_ID,Age,Gender,Education_Level,Income,Joining Designation,Grade,Total Business Value,Quarterly Rating
count,19104.000000,19104.000000,19043.000000,19052.000000,19104.000000,19104.000000,19104.000000,19104.000000,1.910400e+04,19104.000000
mean,9551.500000,1415.591133,34.668435,0.418749,1.021671,65652.025126,1.690536,2.252670,5.716621e+05,2.008899
std,5514.994107,810.705321,6.257912,0.493367,0.800167,30914.515344,0.836984,1.026512,1.128312e+06,1.009832
min,0.000000,1.000000,21.000000,0.000000,0.000000,10747.000000,1.000000,1.000000,-6.000000e+06,1.000000
25%,4775.750000,710.000000,30.000000,0.000000,0.000000,42383.000000,1.000000,1.000000,0.000000e+00,1.000000
50%,9551.500000,1417.000000,34.000000,0.000000,1.000000,60087.000000,1.000000,2.000000,2.500000e+05,2.000000
75%,14327.250000,2137.000000,39.000000,1.000000,2.000000,83969.000000,2.000000,3.000000,6.997000e+05,3.000000
max,19103.000000,2788.000000,58.000000,1.000000,2.000000,188418.000000,5.000000,5.000000,3.374772e+07,4.000000


In [10]:
ola.describe(include= 'object')

,MMM-YY,City,Dateofjoining,LastWorkingDate
count,19104,19104,19104,1616
unique,24,29,869,493
top,01/01/19,C20,23/07/15,29/07/20
freq,1022,1008,192,70


In [11]:
ola.drop(columns = 'Unnamed: 0', axis=1, inplace=True)

In [12]:
ola.nunique()

,0
MMM-YY,24
Driver_ID,2381
Age,36
Gender,2
City,29
Education_Level,3
Income,2383
Dateofjoining,869
LastWorkingDate,493
Joining Designation,5


In [13]:
ola.isna().sum()

,0
MMM-YY,0
Driver_ID,0
Age,61
Gender,52
City,0
Education_Level,0
Income,0
Dateofjoining,0
LastWorkingDate,17488
Joining Designation,0


###**DATA PROCESSING AND FEATURE ENGINEERING**

In [16]:
ola.head()

,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,01/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,02/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,03/01/19,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,11/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
4,12/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1


In [15]:
ola1 = ola.copy(deep=True)

In [17]:
## Target variable creation: Create a column called target which tells whether the driver has left the company

## diver whose last working day is present will have 1
first = (ola1.groupby('Driver_ID').agg({'LastWorkingDate':'last'})['LastWorkingDate'].isna()).reset_index()
first['LastWorkingDate'].replace({True:1, False:0}, inplace = True)
first.rename(columns={'LastWorkingDate': 'target'}, inplace = True)
first.head()

,Driver_ID,target
0,1,0
1,2,1
2,4,0
3,5,0
4,6,1


In [18]:
# Create column which tells whether the quaterly rating has increased for that driver -
# for those whose quaterly rating has increased we assign the value 1
QR1 = (ola1.groupby('Driver_ID').agg({'Quarterly Rating': 'first'})['Quarterly Rating']).reset_index()
QR2 = (ola1.groupby('Driver_ID').agg({'Quarterly Rating': 'last'})['Quarterly Rating']).reset_index()

In [19]:
QR1.shape,QR2.shape

((2381, 2), (2381, 2))

In [20]:
QR1.isna().sum(),QR2.isna().sum()

(Driver_ID           0
 Quarterly Rating    0
 dtype: int64,
 Driver_ID           0
 Quarterly Rating    0
 dtype: int64)

In [22]:
first = first.merge(QR1, on='Driver_ID')
first = first.merge(QR2, on = 'Driver_ID')

In [23]:
first.head()

,Driver_ID,target,Quarterly Rating_x,Quarterly Rating_y
0,1,0,2,2
1,2,1,1,1
2,4,0,1,1
3,5,0,1,1
4,6,1,1,2


In [34]:
first['Promotion'] = np.where(first['Quarterly Rating_x'] == first['Quarterly Rating_y'], 0, 1)

In [35]:
#Create a column which tells whether the monthly income has increases for that driver -
# for those whose monthly income has increased we assign the value 1
incm1 = (ola1.groupby('Driver_ID').agg({'Income':'first'})['Income']).reset_index()
incm2 = (ola1.groupby('Driver_ID').agg({'Income': 'last'})['Income']).reset_index()

In [36]:
incm1.shape,incm2.shape

((2381, 2), (2381, 2))

In [37]:
incm1.isna().sum(), incm2.isna().sum()

(Driver_ID    0
 Income       0
 dtype: int64,
 Driver_ID    0
 Income       0
 dtype: int64)

In [38]:
first = first.merge(incm1, on = 'Driver_ID')
first = first.merge(incm2, on = 'Driver_ID')

MergeError: Passing 'suffixes' which cause duplicate columns {'Income_x'} is not allowed.

In [29]:
first.head()

,Driver_ID,target,Quarterly Rating_x,Quarterly Rating_y,Income_x,Income_y
0,1,0,2,2,57387,57387
1,2,1,1,1,67016,67016
2,4,0,1,1,65603,65603
3,5,0,1,1,46368,46368
4,6,1,1,2,78728,78728


In [30]:
first['Raise'] = np.where(first['Income_x'] == first['Income_y'], 0, 1)

In [39]:
first.head()

,Driver_ID,target,Quarterly Rating_x,Quarterly Rating_y,Income_x,Income_y,Raise,Promotion,Income
0,1,0,2,2,57387,57387,0,0,57387
1,2,1,1,1,67016,67016,0,0,67016
2,4,0,1,1,65603,65603,0,0,65603
3,5,0,1,1,46368,46368,0,0,46368
4,6,1,1,2,78728,78728,0,1,78728


In [40]:
first.tail()

,Driver_ID,target,Quarterly Rating_x,Quarterly Rating_y,Income_x,Income_y,Raise,Promotion,Income
2376,2784,1,3,4,82815,82815,0,1,82815
2377,2785,0,1,1,12105,12105,0,0,12105
2378,2786,0,2,1,35370,35370,0,1,35370
2379,2787,0,2,1,69498,69498,0,1,69498
2380,2788,1,1,2,70254,70254,0,1,70254


In [41]:
first= first[['Driver_ID','target','Raise','Promotion']]

In [42]:
first.head()

,Driver_ID,target,Raise,Promotion
0,1,0,0,0
1,2,1,0,0
2,4,0,0,0
3,5,0,0,0
4,6,1,0,1


In [45]:
ola1.head()

,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,01/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,02/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,03/01/19,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,11/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
4,12/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1


In [46]:
functions = {
    'MMM-YY' : 'count',
    'Driver_ID': 'first',
    'Age': 'max',
    'Gender': 'last',
    'City': 'last',
    'Education_Level': 'last',
    'Dateofjoining': 'first',
    'Grade': 'last',
    'Total Business Value': 'sum',
    'Income':'sum',
    'Joining Designation' : 'last',
    'Grade': 'last',
    'Quarterly Rating' : 'first'
}
ola1 = ola1.groupby([ola1['Driver_ID']]).aggregate(functions)
ola1['month'] = pd.to_datetime(ola1['Dateofjoining']).dt.month
ola1['year'] = pd.DatetimeIndex(ola1['Dateofjoining']).year
ola1.rename(columns = {'MMM-YY' : 'Reportings'}, inplace = True)

In [47]:
ola1.reset_index(drop=True, inplace = True)
ola1 = ola1.merge(first, on = 'Driver_ID')
ola1.head()

,Reportings,Driver_ID,Age,Gender,City,Education_Level,Dateofjoining,Grade,Total Business Value,Income,Joining Designation,Quarterly Rating,month,year,target,Raise,Promotion
0,3,1,28.0,0.0,C23,2,24/12/18,1,1715580,172161,1,2,12,2018,0,0,0
1,2,2,31.0,0.0,C7,2,11/06/20,2,0,134032,2,1,11,2020,1,0,0
2,5,4,43.0,0.0,C13,2,12/07/19,2,350000,328015,2,1,12,2019,0,0,0
3,3,5,29.0,0.0,C9,0,01/09/19,1,120360,139104,1,1,1,2019,0,0,0
4,5,6,31.0,1.0,C11,1,31/07/20,3,1265000,393640,3,1,7,2020,1,0,1


In [48]:
import regex
ola1['Age'] = ola1['Age'].astype('int64')
ola1['Cities'] = ola1['City'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis = 1).astype(int)

In [49]:
ola1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381 entries, 0 to 2380
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Reportings            2381 non-null   int64  
 1   Driver_ID             2381 non-null   int64  
 2   Age                   2381 non-null   int64  
 3   Gender                2381 non-null   float64
 4   City                  2381 non-null   object 
 5   Education_Level       2381 non-null   int64  
 6   Dateofjoining         2381 non-null   object 
 7   Grade                 2381 non-null   int64  
 8   Total Business Value  2381 non-null   int64  
 9   Income                2381 non-null   int64  
 10  Joining Designation   2381 non-null   int64  
 11  Quarterly Rating      2381 non-null   int64  
 12  month                 2381 non-null   int32  
 13  year                  2381 non-null   int32  
 14  target                2381 non-null   int64  
 15  Raise                

In [51]:
ola1.drop(columns= ['Dateofjoining','City'], axis= 1, inplace=True)
ola1['Gender'].replace({'M':0,'F': 1}, inplace=True)
ola1['Gender'] = ola1['Gender'].astype('int64')

In [52]:
ola1.head()

,Reportings,Driver_ID,Age,Gender,Education_Level,Grade,Total Business Value,Income,Joining Designation,Quarterly Rating,month,year,target,Raise,Promotion,Cities
0,3,1,28,0,2,1,1715580,172161,1,2,12,2018,0,0,0,23
1,2,2,31,0,2,2,0,134032,2,1,11,2020,1,0,0,7
2,5,4,43,0,2,2,350000,328015,2,1,12,2019,0,0,0,13
3,3,5,29,0,0,1,120360,139104,1,1,1,2019,0,0,0,9
4,5,6,31,1,1,3,1265000,393640,3,1,7,2020,1,0,1,11


In [53]:
sum(ola1.isna().sum())

0

In [54]:
ola1.describe().T

,count,mean,std,min,25%,50%,75%,max
Reportings,2381.0,8.023520e+00,6.783590e+00,1.0,3.0,5.0,10.0,24.0
Driver_ID,2381.0,1.397559e+03,8.061616e+02,1.0,695.0,1400.0,2100.0,2788.0
Age,2381.0,3.366317e+01,5.983375e+00,21.0,29.0,33.0,37.0,58.0
Gender,2381.0,4.103318e-01,4.919972e-01,0.0,0.0,0.0,1.0,1.0
Education_Level,2381.0,1.007560e+00,8.162900e-01,0.0,0.0,1.0,2.0,2.0
Grade,2381.0,2.096598e+00,9.415218e-01,1.0,1.0,2.0,3.0,5.0
Total Business Value,2381.0,4.586742e+06,9.127115e+06,-1385530.0,0.0,817680.0,4173650.0,95331060.0
Income,2381.0,5.267603e+05,6.231633e+05,10883.0,139895.0,292980.0,651456.0,4522032.0
Joining Designation,2381.0,1.820244e+00,8.414334e-01,1.0,1.0,2.0,2.0,5.0
Quarterly Rating,2381.0,1.486350e+00,8.343483e-01,1.0,1.0,1.0,2.0,4.0
